<center>
    
# 3 - Introduction to `Dask-image`
    
</center>

`Dask-image` is a `Dask` sub-package specialized for handling images.

It supports many of the functions from `scipy.ndimage` for `Dask.arrays`

* dask_image.dispatch
* dask_image.imread
* dask_image.ndfilters
* dask_image.ndfourier
* dask_image.ndinterp
* dask_image.ndmeasure
* dask_image.ndmorph

If you are interested, check out the [following example](https://genevievebuckley.github.io/dask-image-talk-2020/), from which this Notebook was partly inspired. 

In this notebook, we will use `Dask-image` for **lazy loading** a large image and **filtering** it.

### 1) Lazy loading with Dask_image.imread

In [ ]:
import os

import dask.array as da
import dask_image.imread

from utils import show_images

filename_pattern = os.path.join("imgs", "hubble", "image-*.png")
tiled_hubble_images = dask_image.imread.imread(filename_pattern)

# Convert image to float
tiled_hubble_images = tiled_hubble_images.astype(float) / 255.0
tiled_hubble_images

In [ ]:
# REMINDER: Dask is lazy by default
from sys import getsizeof
print(f"Size of `dask-image`: {getsizeof(tiled_hubble_images)} bytes")

In [ ]:
show_images(
    [
        tiled_hubble_images[0],
        tiled_hubble_images[5],
        tiled_hubble_images[10],
        tiled_hubble_images[15],
        tiled_hubble_images[20],
    ]
)

### 2) Applying your own custom function to images

Next you'll want to do some image processing, and apply a function to your images.

We'll use a very simple example: converting an RGB image to grayscale. 

To convert our image to grayscale, we'll use the equation to calculate luminance:

```Y = 0.2125 R + 0.7154 G + 0.0721 B```

We'll write the function for this equation as follows:

In [ ]:
def grayscale(rgb):
    result = (rgb[..., 0] * 0.2125) + (rgb[..., 1] * 0.7154) + (rgb[..., 2] * 0.0721)
    return result

As you can see, this operation requires to have a full extra copy of the input image in memory. Many operations require the creation, even temporarily, of array of the same size as the image. This can easily pose a bottleneck in our pipeline!

Let's apply our custom function to the a **single chunk** of our image and visualize the computation graph.

*TIP: Visualizing the computation graph isn't necessary but most of the time it's helpful to know what dask is doing under the hood, and it can also be very useful for debugging problems.*

In [ ]:
hubble_image = tiled_hubble_images[20].persist()
import matplotlib.pyplot as plt
plt.imshow(hubble_image)

In [ ]:
result_single_image = grayscale(hubble_image)
print(result_single_image)
result_single_image.visualize()

In [ ]:
print("Original image dimensions: ", hubble_image.shape)
print("Processed image dimensions:", result_single_image.shape)

show_images(
    [hubble_image, result_single_image],
    titles=["Original image", "Processed image"],
    cmap=[None, "gray"],
)

### Embarrassingly parallel problems

The syntax is identical to apply a function to multiple images or dask chunks. This is an example of an embarrassingly parallel problem, and we see that Dask automatically creates a computation graph for each image chunk.

In [ ]:
result = grayscale(tiled_hubble_images[:5].persist())
print(result)
result.visualize()

In [ ]:
result = grayscale(tiled_hubble_images)
show_images(
    [
        result[0],
        result[5],
        result[10],
        result[15],
        result[20],
    ]
)

### Join processed chunks
We will now concatenate the small image chunks together into the shape of the image. 

This is important when doing convolutions or other analysis that process the neighborhood of each pixel.

This joined image will still be a **lazy array**, but each chunk will access the edge values of the neighboring chunks. 

*Note: We could have done this in the beggining too.*

In [ ]:
# Toy Dataset for visualisation
data = [result[20], result[21]]
toy_combined_image = da.block(data)
toy_combined_image

### Let's do some Dask-image processing

For illustration purposes, let's perform some [edge enhancement](https://www.wikiwand.com/en/Edge_enhancement) of our sky image.

A very simple approach for edge enhancement is the following: 

$$\hat{\mathbf{I}} = (1-\alpha) \mathbf{I} + \alpha \vert  \mathbf{I} - \bar{\mathbf{I}}_{\sigma} \vert $$

Where:

- $\hat{\mathbf{I}}$ is the enhenced image,
- $\mathbf{I}$ is the original image, 
- $\bar{\mathbf{I}}_{\sigma} $ is a smoothed version of $\mathbf{I}$, with smoothing parameter $\sigma$, and
- $\alpha \in [0, 1]$ controls the level of enhancement. 


Let's complete the following function `edge_enhancement(image, sigma)` that uses `dask_image.ndfilters.gaussian_filter`:

```python
def edge_enhancement(image, sigma):
    """    
    Parameters
    ----------
    image: Dask.array (2d)
        Input image to enhance.
    sigma: float
        Smoothing parameter.
    alpha: float [0, 1]
        
        
    Returns
    -------
    out_image: Dask.array (2d)
        Edge enhanced image.
    """
    
    return NotImplementedError
```

In [ ]:
import dask_image.ndfilters

def edge_enhancement(image, sigma, alpha):
    """    
    Parameters
    ----------
    image: Dask.array (2d)
        Input image to enhance.
    sigma: float
        Smoothing parameter.
    alpha: float [0, 1]
        
        
    Returns
    -------
    out_image: Dask.array (2d)
        Edge enhanced image.
    """
    assert sigma > 0
    assert  0 <= alpha <= 1
    smoothed_image = dask_image.ndfilters.gaussian_filter(image, sigma=sigma)
    edge_image = abs(image - smoothed_image)
    edge_enhanced_image = alpha * image + alpha * abs(image - smoothed_image)
    
    return edge_enhanced_image

In [ ]:
edge_enhanced_toy = edge_enhancement(toy_combined_image.persist(), 
                                     sigma=20, 
                                     alpha=0.5)

Now try to match the different parts of the task graph with the edge enhancement equation: 

$$\hat{\mathbf{I}} = (1-\alpha) \mathbf{I} + \alpha \vert  \mathbf{I} - \bar{\mathbf{I}}_{\sigma} \vert $$


In [ ]:
edge_enhanced_toy

In [ ]:
edge_enhanced_toy.visualize(rankdir="LR")

The toy dataset is useful to understand the task graph, and it can also be used to select good values for $\sigma$ and $\alpha$:

In [ ]:
import itertools

alphas = [0.1, 0.5]
sigmas = [1, 10, 20]

for alpha, sigma in itertools.product(alphas, sigmas):
    edge_enhanced_toy = edge_enhancement(toy_combined_image.persist(), 
                                     sigma=sigma, 
                                     alpha=alpha)
    show_images(
        [toy_combined_image, edge_enhanced_toy],
        titles=["Original", fr"Enh ($\sigma={sigma}$, $\alpha={alpha}$)"],
        cmap="cubehelix"
    )

### Final step: run full pipeline on RGB dataset, and save for later. 

In [ ]:
# Enhancing parameters
sigma = 20
alpha = 0.5

# Full Dataset
edge_enhanced_image = []
combined_image = []
# For each RGB
for j in range(3):
    # Join chunks to have access to neighborhoods
    data = [
        [img[..., j] for img in tiled_hubble_images[i * 28 : (i + 1) * 28]]
        for i in range(24)
    ]
    combined_image.append(da.block(data))
    
    # Join chunks to have access to neighborhoods
    edge_enh = edge_enhancement(combined_image[j], sigma=sigma, alpha=alpha)
    edge_enhanced_image.append(edge_enh)

# Convert list to dask array
edge_enhanced_image = da.dstack(edge_enhanced_image)
combined_image = da.dstack(combined_image)
edge_enhanced_image

In [ ]:
show_images(
    [
        combined_image.blocks[0, 20],
        edge_enhanced_image.blocks[0, 20],
        (combined_image-edge_enhanced_image).blocks[0, 20],
    ],
    titles=["Original", fr"Edge Enhanced ($\sigma={sigma}$, $\alpha={alpha}$)", "Difference"],
)

##### Save Data
To simplify data loading in the future, we store this large chunked-array as a [Zarr file](https://zarr.readthedocs.io/en/stable/) using the `dask.array.to_zarr` method.

In [ ]:
edge_enhanced_image.to_zarr(os.path.join("imgs", "hubble_enh.zarr"), overwrite=True)